#  Lexical Resource (LR) for Annotation

#### Imports

In [1]:
# interface
from tqdm import tqdm
from dsutils.de.files import dbg

## Files and filesystem
import os
import json
import csv
import glob

## Data management
import numpy as np
import pandas as pd

## DS tools
import re
from types import NoneType
from dsutils.de.files import describe_csv, get_csv_head, xls_to_csv, get_data_path, get_datafile_path
from dsutils.nlp.language import get_lang_code

[nltk_data] Downloading package words to /home/elubrini/nltk_data...
[nltk_data]   Package words is already up-to-date!


#### Paths

In [2]:
## input
db_paths = dict(
    eppo_com = get_datafile_path('original/2022-09-02_COMMONnames_EPPO_OQ.xlsx'),
    eppo_sci = get_datafile_path('original/2022-09-02_SCIENTIFICnames_EPPO_OQ.xlsx'),
    efsa = get_datafile_path('EFSA-keyword-match/FichierMotsClesMagaliLarenaudie.csv'),
    ncbi = get_datafile_path('taxa+id_full.txt'),
)

glossary_paths = dict(
    efsa_glossary = get_datafile_path('efsa_glossary.csv'),
    eppo_glossary = get_datafile_path('eppo_glossary.csv'),
)

## Output
data_path = get_data_path()
full_eppo_path = os.path.join(data_path, 'eppo_glossary.csv')
annotation_path = os.path.join(data_path, 'LRs_for_annotation')
anno_gloss_path = os.path.join(data_path, 'annotation_glossary.csv')
LR_filepath = 'LR.csv'

/home/elubrini/GitHub/bio-corpus-translation/venv/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/elubrini/GitHub/bio-corpus-translation/venv/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


#### Definitions

Defining functions for preprocessing of original data

In [3]:
def PHT_to_lower_taxon(PHT_code):
    taxon = re.sub(r'(?<!^)(?=[A-Z])', ' ', PHT_code[:-4]).lower()
    return taxon
# print('AcalymmaVittatum-PHT:',PHT_to_lower_taxon('AcalymmaVittatum-PHT')) # test
assert PHT_to_lower_taxon('AcalymmaVittatum-PHT') == 'acalymma vittatum'

def to_pattern(syn):
    r = syn.replace('+', ' ')
    r = r.replace('_', '\\w')
    if r.endswith('%'):
        r = r[:-1] + '\\w*'
    r = r.replace('% ', '\\w* ')
    r = r.replace('%', '\\S*\\s?')
    return r
assert to_pattern(r'chrysom_le%+ray_e+du+concombre') == 'chrysom\wle\w* ray\we du concombre'

#### Parameters

In [4]:
ncbi_sep = '\t'
ncbi_header = ['synonym','taxid','sci_name','tax_path','POS','rank','undefined','also_undefined']

## name of the desired coluimns in the LR
anno_gloss_column_names = ['EPPO_CODE', 'EFSA_PHT', 'NCBI_TAXID', # 3 partial keys to relative dataset
                           'preferred_name', 'synonym', 're',# terms
                           'ds_language', 'match_language'] # languages

In [5]:
## column in each of the original datasets corresponding to the output LR column 
cc = dict(
    eppo_com = ['CodeEOPP', None, None,
                'PreferredName', 'CommonName', None,
                ['Language', get_lang_code], None], # [obj (col name), fun] tuple when information is extracted via a function
    eppo_sci = ['CodeEOPP', None, None,
                'PreferredName', 'OtherScientificNames', None,
                None, None],
    efsa = [None, 'Category (pest name)', None,
                ['Category (pest name)', PHT_to_lower_taxon], None,  ['Keywords', to_pattern],
                None, 'Unnamed: 3'],
    ncbi = [None, None, 'taxid',
            'sci_name', 'synonym', None,
            None, None],
    )

#### Check Data

In [6]:
for path in db_paths.values():
    display(describe_csv(path))
pd.read_csv(db_paths['ncbi'], names=ncbi_header, index_col=0)

,CommonName,Language,CodeEOPP,PreferredName,AuthorityPreferredName
count,1276,1276,1324,1324,1142
unique,1235,38,239,239,169
top,citrus leprosis,English,HELIZE,Helicoverpa zea,(Boddie)
freq,4,477,30,30,30


,OtherScientificNames,Authority,CodeEOPP,PreferredName,AuthorityPreferredName
count,616,453,664,664,502
unique,611,324,239,239,169
top,Tomato leaf curl New Delhi begomovirus,Corbett,BEMITA,Bemisia tabaci,(Gennadius)
freq,2,6,16,16,16


,Category (pest name),Keywords,Unnamed: 2,Unnamed: 3
count,7558,7558,21,3
unique,1168,7393,5,3
top,Cronartium-PHT,margarodes,Au/Cabi,Fr
freq,85,3,16,1


,Unnamed: 0
count,100.000000
mean,49.500000
std,29.011492
min,0.000000
25%,24.750000
50%,49.500000
75%,74.250000
max,99.000000


,synonym,taxid,sci_name,tax_path,POS,rank,undefined,also_undefined
NaN,Bacteria,ncbi:2,Bacteria.1,/ncbi:1/ncbi:131567/ncbi:2,NP,superkingdom,Unnamed: 6,Unnamed: 7
0.0,bacteria,ncbi:2,Bacteria,/ncbi:1/ncbi:131567/ncbi:2,NNS,superkingdom,NaN,NaN
1.0,eubacteria,ncbi:2,Bacteria,/ncbi:1/ncbi:131567/ncbi:2,NN,superkingdom,NaN,NaN
2.0,Monera,ncbi:2,Bacteria,/ncbi:1/ncbi:131567/ncbi:2,NP,superkingdom,NaN,NaN
3.0,Procaryotae,ncbi:2,Bacteria,/ncbi:1/ncbi:131567/ncbi:2,NP,superkingdom,NaN,NaN
...,...,...,...,...,...,...,...,...
95.0,Dictyoglomus Saiki et al. 1985,ncbi:13,Dictyoglomus,/ncbi:1/ncbi:131567/ncbi:2/ncbi:68297/ncbi:203...,NP,genus,NaN,NaN
96.0,Dictyoglomus,ncbi:13,Dictyoglomus,/ncbi:1/ncbi:131567/ncbi:2/ncbi:68297/ncbi:203...,NP,genus,NaN,NaN
97.0,Dictyoglomi,ncbi:13,Dictyoglomus,/ncbi:1/ncbi:131567/ncbi:2/ncbi:68297/ncbi:203...,NP,genus,NaN,NaN
98.0,ATCC 35947,ncbi:14,Dictyoglomus thermophilum,/ncbi:1/ncbi:131567/ncbi:2/ncbi:68297/ncbi:203...,NP,species,ncbi:14,Dictyoglomus thermophilum


# Create LR

### (1) Preprocess original datasets

Extract relevant information from each dataset and store it in table form (one `csv` per original dataset)

In [7]:
## Display examples of input datasets
display(pd.read_csv(db_paths['eppo_sci']).describe())
pd.read_csv(db_paths['eppo_com']).describe()

,OtherScientificNames,Authority,CodeEOPP,PreferredName,AuthorityPreferredName
count,616,453,664,664,502
unique,611,324,239,239,169
top,Tomato leaf curl New Delhi begomovirus,Corbett,BEMITA,Bemisia tabaci,(Gennadius)
freq,2,6,16,16,16


,CommonName,Language,CodeEOPP,PreferredName,AuthorityPreferredName
count,1276,1276,1324,1324,1142
unique,1235,38,239,239,169
top,citrus leprosis,English,HELIZE,Helicoverpa zea,(Boddie)
freq,4,477,30,30,30


In [8]:

db = dict() # create dictionary of databases

for db_name, columns in cc.items(): # loop over db names and respective relevant columns
    db[db_name] = dict()    # new database in db dictionary
    cc_zip = zip(columns, anno_gloss_column_names)  # match source-db column names to final-db column names

    for orig_col, targ_col in cc_zip: # loop over matches

        if isinstance(orig_col, str): # if only column name specified, take column as is
            def fun(x):
                return x
        elif isinstance(orig_col,NoneType): # else, if column must not to be selected, continue
            continue
        else:    # else, if a preprocessing function has been specified, use it when extracting column
            fun = orig_col[1]
            orig_col = orig_col[0]
        # sep = tab (for ncbi)
        if 'ncbi' in db_name:
            ds_col = pd.read_csv(db_paths['ncbi'], names=ncbi_header, index_col=0)[orig_col]
        else:    
            ds_col = pd.read_csv(db_paths[db_name])[orig_col] 
            
        db[db_name][targ_col] = list(map(fun, ds_col)) # new_db col = old_db col
    
    pd.DataFrame.from_dict(db[db_name]).to_csv(os.path.join(annotation_path, db_name+'_table.csv'), index=False)


### (2) Join Datasets

In [9]:
prep_dbs = dict()
for db_name in cc.keys():
    prep_dbs[db_name] = pd.read_csv(os.path.join(annotation_path, db_name + '_table.csv'))

#### Base new DB off NCBI

In [10]:
base_db = prep_dbs['ncbi']

structured_db = base_db.copy()
for col_name in anno_gloss_column_names:
    if col_name not in structured_db.columns:
        structured_db[col_name] = [None]*len(structured_db) # create empty column with desired col name
display(structured_db.head())

,NCBI_TAXID,preferred_name,synonym,EPPO_CODE,EFSA_PHT,re,ds_language,match_language
0,ncbi:2,Bacteria.1,Bacteria,None,None,None,None,None
1,ncbi:2,Bacteria,bacteria,None,None,None,None,None
2,ncbi:2,Bacteria,eubacteria,None,None,None,None,None
3,ncbi:2,Bacteria,Monera,None,None,None,None,None
4,ncbi:2,Bacteria,Procaryotae,None,None,None,None,None


df1 = pd.DataFrame([[3,4],[5,6]], columns=['a','b'])
df2 = pd.DataFrame([[5,6],[5,6]], columns=['a','c'])
display(df1)
pd.concat([df1,df2], ignore_index=True)

#### Add EPPO info to final DB

In [11]:
db_names = list(str(x) for x in prep_dbs.keys())

In [12]:
# copy into a new db to which eppo information will be added 
ncbi_eppo_db = structured_db.copy()

## open both eppo (sci and com) dbs and concatenate them
prep_eppo_dbs = [prep_dbs[db_name] for db_name in db_names if 'eppo' in db_name]
prep_eppo_db = pd.concat(prep_eppo_dbs, ignore_index=True)
display(prep_eppo_db.head())

,EPPO_CODE,preferred_name,synonym,ds_language
0,LIBEAF,'Candidatus Liberibacter africanus',greening of citrus,en
1,LIBEAF,'Candidatus Liberibacter africanus',greening des agrumes,fr
2,LIBEAF,'Candidatus Liberibacter africanus',virescence des agrumes,fr
3,LIBEAF,'Candidatus Liberibacter africanus',enverdecimiento de los cítricos,es
4,LIBEAM,'Candidatus Liberibacter americanus',Brazilian citrus greening,en


In [13]:
## loop over syns in EPPO db
for prep_eppo_db_i in tqdm(range(len(prep_eppo_db))):
    for ncbi_eppo_db_i in range(len(ncbi_eppo_db)):
        match_found = False
        
        ## if EPPO syn == structured_db.syn: add language, and key to row
        if prep_eppo_db['synonym'][prep_eppo_db_i] == ncbi_eppo_db['synonym'][ncbi_eppo_db_i]:
            ncbi_eppo_db['ds_language'][ncbi_eppo_db_i] = prep_eppo_db['ds_language'][prep_eppo_db_i]
            ncbi_eppo_db['EPPO_CODE'][ncbi_eppo_db_i] = prep_eppo_db['EPPO_CODE'][prep_eppo_db_i]
            match_found = True
        
    ## else, create new row with: sci_name, syn, lang, key
    if not match_found:
        sci_name = prep_eppo_db['preferred_name'][prep_eppo_db_i]
        syn = prep_eppo_db['synonym'][prep_eppo_db_i]
        lang = prep_eppo_db['ds_language'][prep_eppo_db_i]
        key = prep_eppo_db['EPPO_CODE'][prep_eppo_db_i]
        
        new_eppo_row = pd.DataFrame([[sci_name, syn, lang, key]], columns=['preferred_name', 'synonym', 'ds_language', 'EPPO_CODE'])
        
        ncbi_eppo_db = pd.concat([ncbi_eppo_db, new_eppo_row], ignore_index=True)
    else:
        dbg(prep_eppo_db['preferred_name'][prep_eppo_db_i], len(ncbi_eppo_db))

## loop over sci_names in EPPO db
for prep_eppo_db_i in tqdm(range(len(prep_eppo_db))):
    for ncbi_eppo_db_i in range(len(ncbi_eppo_db)):
        
        ## if EPPO sci_name == structured_db.syn: add language, and key
        if prep_eppo_db['synonym'][prep_eppo_db_i] == ncbi_eppo_db['preferred_name'][ncbi_eppo_db_i]:
            ncbi_eppo_db['ds_language'][ncbi_eppo_db_i] = prep_eppo_db['ds_language'][prep_eppo_db_i]
            ncbi_eppo_db['EPPO_CODE'][ncbi_eppo_db_i] = prep_eppo_db['EPPO_CODE'][prep_eppo_db_i]
    
            match_found = True
        
    ## else, create new row with: sci_name, syn, lang, key
    if not match_found and prep_eppo_db['preferred_name'][prep_eppo_db_i] not in ncbi_eppo_db['preferred_name']:
        syn = sci_name = prep_eppo_db['preferred_name'][prep_eppo_db_i]
        lang = prep_eppo_db['ds_language'][prep_eppo_db_i]
        key = prep_eppo_db['EPPO_CODE'][prep_eppo_db_i]
        
        new_eppo_row = pd.DataFrame([[sci_name, syn, lang, key]], columns=['preferred_name', 'synonym', 'ds_language', 'EPPO_CODE'])
        
        ncbi_eppo_db = pd.concat([ncbi_eppo_db, new_eppo_row], ignore_index=True)

 22%|██▏       | 445/1988 [00:10<00:43, 35.25it/s]

540sci_name: Bemisia tabaci


 27%|██▋       | 545/1988 [00:13<00:52, 27.52it/s]

640sci_name: Helicoverpa zea


 30%|██▉       | 593/1988 [00:15<00:56, 24.50it/s]

...

 63%|██████▎   | 1255/1988 [00:38<00:27, 26.72it/s]

1338sci_name: Aphis citricidus


 64%|██████▍   | 1279/1988 [00:39<00:26, 27.23it/s]

1361sci_name: Bursaphelenchus xylophilus


100%|██████████| 1988/1988 [03:03<00:00, 10.86it/s]


#### Add EFSA info to final DB

In [14]:
# copy into a new db to which eppo information will be added 
final_db = ncbi_eppo_db.copy()

## open both eppo (sci and com) dbs and concatenate them
prep_efsa_dbs = [prep_dbs[db_name] for db_name in db_names if 'efsa' in db_name]
prep_efsa_db = pd.concat(prep_efsa_dbs, ignore_index=True)
display(prep_efsa_db.head())

,EFSA_PHT,preferred_name,re,match_language
0,AcaloleptaSejuncta-PHT,acalolepta sejuncta,acalolepta sejuncta,NaN
1,AcalymmaVittatum-PHT,acalymma vittatum,acalymma vittata,NaN
2,AcalymmaVittatum-PHT,acalymma vittatum,acalymma vittatum,NaN
3,AcalymmaVittatum-PHT,acalymma vittatum,chrysom\wle\w* ray\we du concombre,Fr
4,AcalymmaVittatum-PHT,acalymma vittatum,cistela melanocephala,NaN


In [15]:
## loop over res i n EFSA db
for prep_efsa_db_i in tqdm(range(len(prep_efsa_db))):
    for final_db_i in range(len(final_db)):
        pattern = prep_efsa_db['re'][prep_efsa_db_i]
        syn = final_db['synonym'][final_db_i]
        
        ## if re matches a syn, add: re, efsa key, language if None
        if re.match(str(pattern), str(syn)):
            final_db['re'][final_db_i] = prep_efsa_db['re'][prep_efsa_db_i]
            final_db['EFSA_PHT'][final_db_i] = prep_efsa_db['EFSA_PHT'][prep_efsa_db_i]
            final_db['match_language'][final_db_i] = prep_efsa_db['match_language'][prep_efsa_db_i]
            match_found = True
        
    ## else, if no matches, add new line = re, syn, key, language
    if not match_found:
        pattern = prep_efsa_db['re'][prep_efsa_db_i]
        #syn = prep_efsa_db['synonym'][prep_efsa_db_i]
        key = prep_efsa_db['EFSA_PHT'][prep_efsa_db_i]
        lang = prep_efsa_db['match_language'][prep_efsa_db_i]
        
        new_efsa_row = pd.DataFrame([[pattern, syn, lang, key]], columns=['re', 'syn', 'match_language', 'EFSA_PHT'])
        
        final_db = pd.concat(final_db, new_efsa_row, ignore_index=True)
            
final_db.head()

100%|██████████| 7558/7558 [13:51<00:00,  9.09it/s]


,NCBI_TAXID,preferred_name,synonym,EPPO_CODE,EFSA_PHT,re,ds_language,match_language
0,ncbi:2,Bacteria.1,Bacteria,None,None,None,None,None
1,ncbi:2,Bacteria,bacteria,None,None,None,None,None
2,ncbi:2,Bacteria,eubacteria,None,None,None,None,None
3,ncbi:2,Bacteria,Monera,None,None,None,None,None
4,ncbi:2,Bacteria,Procaryotae,None,None,None,None,None


In [16]:
final_db.to_csv(LR_filepath)